# Startegia in tempo reale

In [1]:
# osservazione: la classe Trader la lascio qua perche faro una classe figlia e cambiero soltanto il necessario
# Osservazione: lunita eh gia inserita nella classe, bisogna 1.inserire quantita un euro e 2.creare una strutura che
# controlla gli inputs e, se entrambi vengono dati dall'utente, la strutura "passa avanti" solo la qtita euro.
# 3. se viene scelta la quantita in euro, bisogna calcolare le units: qtitaEUR*c.iloc[-2]*. S
# Magari sarebbe anche interessante fare in maniera che il codice esegue il massimo di units e quantita euro, 
# per evvitare perdite extreme in caso di volatilita' extrema. Idealmente prenderemmo i due prezzi di apertura 
# Aasb/Abid e faremmo una media. *Siccome io non posso comprare/vendere di sera a prezzo chiusura 
# perche il mercado eh chiuso, prendo prezzo chiusura del penultimo giorno disponibile

In [2]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import tpqoa
import math

In [3]:
# non ricalcoli ogni volta il logret e strategia per i dati, ma soltanto per l'ultima candela
# lidea qua eh creare un boolean che informera al metodo se deve calcolare per tutti dati o solo candella
# raw data contiene dato del passato, degli ultimi 5 giorni e tickdata contiene dati di oggi.

class Trader(tpqoa.tpqoa): # classe figlia della tpqoa del pachetto tpqoa che gestisce le apis

#modifico construtore per inserire dataframe dove inserisco i dati 
    def __init__(self,conf_file, instrument,period, window,units):
        super().__init__(conf_file)
        self.tickData=pd.DataFrame()  # 1 tick = 1 trasazione, viene maniera disordinata
        self.rawData=None # tickdata ragruppato
        self.instrument=instrument
        self.period=period
        # il primo ultimo periodo eh il periodo in cui costruisco la classe
        self.ultimoPeriodo= None
        self.data=None
        self.window=window    
        self.units=units # unita che compriamo vendiamo ad ogni cambio di posizione. meglio: comp,vend quant. fisso denaro
        self.posizione=0 #posizione istantanea,corrente 
        self.profits=[]
        
#parte che carica i dati storici
# dipende da come scegli period
    def get_most_recent(self,days=5):
        while True:
            now=datetime.utcnow()
            now = now -timedelta(microseconds=now.microsecond) #mi crea now togliendo i microsecondi
            yesterday = now - timedelta(days=days) #none ieri, sono days giorni prima
            df=self.get_history(self.instrument,start=yesterday,end=now,granularity="S5",price="M",localize=False).c.to_frame() 
            # attenzione se self.period DEVE ESSERE= o + grande di 5 secondi
            df.rename(columns={"c":"EUR_USD"},inplace=True)
            #butto via lultimo periodo perche potrebbe essere incompleto
            self.rawData=df.resample(self.period,label="right").last().dropna().iloc[:-1] 
            #self.period devve essere piu grande di 5 
            self.ultimoPeriodo=self.rawData.index[-1] #potrebbe essere incompleto
            if (pd.to_datetime(datetime.utcnow()).tz_localize("UTC")-self.ultimoPeriodo)<pd.to_timedelta(self.period):
                break
            else:
                print("Purtroppo mi devo riscaricare i dati :(")
                time.sleep(2)
# attenzione a self.period<=granularity. se self.period>granularity non riesco a ressamplare 
# i dati perche non ce li ho
    
    
#eredita tutti i metodi, ridefiniamo on sucess
#mi mette le cose in un dataframe, mid mi devo calcolare io, mi fornisce pronto solo per dati storici
#il dataframe ha una sola riga che eh il tick che ha apppena arrivato
#print(time, bid, ask)
    def on_success(self,time,bid,ask): 
        print(self.ticks, end=" ") #ogni volta che mi arriva un tick io mi creo DF che ha solo una rigga
        df=pd.DataFrame({self.instrument:(bid+ask)/2},
                       index=[pd.to_datetime(time)])
        self.tickData=self.tickData.append(df) #appendo sto DF allaltro
        tickCorrente=pd.to_datetime(time) 
        if (tickCorrente-self.ultimoPeriodo)>pd.to_timedelta(self.period): #ho chiuso blochetto da 5 secondi -> oko candella?
            self.resampleJoin()        #e devo ressamplare
            self.defineStrategy()      # ** OKO 
            self.executeTrade()
 # ressample join chiamo quando ho appena chiuso una candella e sono un tick avante, il tick corrente eh oltre la chiusura 
# dalla candella . se guardo sopra faccio resample excludendo lultimo tick
    
    #la prima riga fa resample di tutti i dati, la seconda riga buta via tutti i dati che ho utilizzati fino ad adesso
    # quindi il programa fa gia 
    def resampleJoin(self): # fa resample di tutti dati, daglinizio fino ad adesso
        self.rawData= self.rawData.append(self.tickData.resample(self.period,label="right").last().ffill().iloc[:-1])
        self.tickData=self.tickData.iloc[-1:] #tickdata butta via tranne lultimo, quindi nel ressample prossimo
        #il tickdata no li trova piu 
        self.ultimoPeriodo=self.rawData.index[-1]
        
    def defineStrategy(self):  # oko onlyone
        df = self.rawData.copy()
        df["logRet"]=np.log((df[self.instrument])/(df[self.instrument].shift(1)))
        df["posizione"]= -np.sign(df.logRet.rolling(self.window).mean()) #**
        self.data=df.copy()
#**ogni volta mi ricalcolo log ret e posizione per tutti gli interavali (candelle), 
#cosa che quando lalgoritmo per lungo tempo, diventa molto inefficiente, bastarebbe calcolarlo solo per lultimo
#*soluzione: renderlo vettoriale. ogni volta che viene una candella nova, calcolare posizione e logret solo per lei

#va a guardare lultima posizione
    def executeTrade(self):
        if self.data.posizione.iloc[-1]==1: #andare lungo
            if self.posizione==0:
                order = self.create_order(instrument=self.instrument, units=self.units,suppress=True,ret=True)
                self.report_order(order, "vado lungo")
            elif self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=2*self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo posizione corta e vado lungo")
            self.posizione=1
        elif self.data.posizione.iloc[-1]==-1: #andare corto
            if self.posizione==0:
                order = self.create_order(instrument=self.instrument, units=-self.units,suppress=True,ret=True)
                self.report_order(order, "vado corto")
            elif self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=-2*self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo posizione lunga e vado corto")
            self.posizione=-1
        elif self.data.posizione.iloc[-1]==0:
            if self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo corto")
            elif self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=-self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo lungo")
            self.posizione=0
            
    def report_order(self,order,message): #mai fare com simple returns ok
        time=order["time"]
        units= order["units"]
        price=order["price"]
        pl=float(order["pl"])
        self.profits.append(pl)
        cumpl=sum(self.profits)
        print("\n---------------------------------------------")
        print(time, "|",message)
        print("quantita ", units, "| prezzo: ", price,"| profit/loss ",pl, "| profitto totale= ", cumpl )
        print("-----------------------------------------------")
    
    # per caricare dati attuali
    def stream_data(self, instrument=None, stop=None, ret=False, callback=None):
        if instrument is None:
            instrument=self.instrument
        super().stream_data(instrument,stop,ret,callback)
        if self.posizione!=0:
            close_all=self.create_order(self.instrument,units=-self.posizione*self.units,suppress=True,ret=True)
            self.report_order(close_all,"chiudo tutto, ho finito")
            self.posizione=0      
        

### Modifica la classe Trader dando la possibilità all'utente di inserire sia le unità di strumento che si vuole acquistare, che il quantitativo di euro che si vuole spendere. Se vengono inserite entrambe allora la classe userà il quantitativo di euro. In seguito il programma avrà a disposizione quel quantitativo di euro e ad ogni trade comprerà/venderà solo quel quantitativo di euro in base all'ultimo prezzo disponibile ask/bid. Quindi ad ogni trade il quantitativo comprato/venduto potrebbe variare leggemente. Per verificare consiglio di utilizzare BTC_USD dato che Bitcoin ha un prezzo molto volatile.

In [4]:

class TraderV2(Trader): 

    def __init__(self,conf_file, instrument,period, window,units=None,qtyEUR=None):
        super().__init__(conf_file, instrument,period, window,units)
        #super().__init__(conf_file)
        self.qtyEUR=qtyEUR
        

    
#eredita tutti i metodi, ridefiniamo on sucess
#mi mette le cose in un dataframe, mid mi devo calcolare io, mi fornisce pronto solo per dati storici
#il dataframe ha una sola riga che eh il tick che ha apppena arrivato

    def on_success(self,time,bid,ask): #viene chiamata nel streamdata dalla classe madre
        print(self.ticks, end=" ") #ogni volta che mi arriva un tick io mi creo DF che ha solo una rigga
        df=pd.DataFrame({self.instrument:(bid+ask)/2},
                       index=[pd.to_datetime(time)])
        self.tickData=self.tickData.append(df) 
        tickCorrente=pd.to_datetime(time) 
        if (tickCorrente-self.ultimoPeriodo)>pd.to_timedelta(self.period): #ho chiuso blochetto da 5 secondi -> oko candella?
            self.resampleJoin()
            self.defineStrategy()
            if (self.qtyEUR is not None):
                self.executeTrade(True) # oko here change 
            else:
                self.executeTrade(False)


#va a guardare lultima posizione
    def executeTrade(self, hasQtyEur): # oko here change
        if (hasQtyEur): #attenzione bisogna capire quale prezzo ci serve qua:
            self.units= math.floor(self.qtyEUR*self.data.EUR_USD.iloc[-2])  # *
        if self.data.posizione.iloc[-1]==1: #andare lungo
            if self.posizione==0:
                order = self.create_order(instrument=self.instrument, units=self.units,suppress=True,ret=True)
                self.report_order(order, "vado lungo")
            elif self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=2*self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo posizione corta e vado lungo")
            self.posizione=1
        elif self.data.posizione.iloc[-1]==-1: #andare corto
            if self.posizione==0:
                order = self.create_order(instrument=self.instrument, units=-self.units,suppress=True,ret=True)
                self.report_order(order, "vado corto")
            elif self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=-2*self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo posizione lunga e vado corto")
            self.posizione=-1
        elif self.data.posizione.iloc[-1]==0:
            if self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo corto")
            elif self.posizione==-1:
                order = self.create_order(instrument=self.instrument, units=-self.units,suppress=True,ret=True)
                self.report_order(order, "chiudo lungo")
            self.posizione=0


In [5]:
tV23 = TraderV2("oanda.cfg","EUR_USD","20S",1,qtyEUR=10000)
print(datetime.utcnow()) 
tV23.get_most_recent() 
print(datetime.utcnow())
tV23.stream_data(tV23.instrument,stop=20)

2021-11-23 16:08:18.913075
Purtroppo mi devo riscaricare i dati :(
2021-11-23 16:08:35.049262
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

In [6]:
tV23.data

,EUR_USD,logRet,posizione
2021-11-18 16:08:40+00:00,1.13500,NaN,NaN
2021-11-18 16:09:00+00:00,1.13498,-0.000018,1.0
2021-11-18 16:09:20+00:00,1.13500,0.000018,-1.0
2021-11-18 16:09:40+00:00,1.13516,0.000141,-1.0
2021-11-18 16:10:00+00:00,1.13504,-0.000106,1.0
...,...,...,...
2021-11-23 16:07:20+00:00,1.12636,0.000062,-1.0
2021-11-23 16:07:40+00:00,1.12644,0.000071,-1.0
2021-11-23 16:08:00+00:00,1.12644,0.000000,-0.0
2021-11-23 16:08:20+00:00,1.12648,0.000036,-1.0


In [7]:
tV23.units

11264

In [8]:
tV23.qtyEUR

10000

In [9]:
tV2 = TraderV2("oanda.cfg","EUR_USD","20S",1,11286)
print(datetime.utcnow()) # chiedo la data 
tV2.get_most_recent() 
print(datetime.utcnow())
tV2.stream_data(tV2.instrument,stop=20)

2021-11-23 16:08:50.442697
2021-11-23 16:08:55.506781
1 2 
---------------------------------------------
2021-11-23T16:09:01.167403035Z | vado corto
quantita  -11286.0 | prezzo:  1.12653 | profit/loss  -17.8205 | profitto totale=  -17.8205
-----------------------------------------------
3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
---------------------------------------------
2021-11-23T16:09:17.190850752Z | chiudo tutto, ho finito
quantita  11286.0 | prezzo:  1.1266 | profit/loss  0.0 | profitto totale=  -17.8205
-----------------------------------------------


In [10]:
tV2.units

11286

In [11]:
tV3 = TraderV2("oanda.cfg","EUR_USD","30S",1,qtyEUR=500)
print(datetime.utcnow()) # chiedo la data 
tV3.get_most_recent() 
print(datetime.utcnow())
tV3.stream_data(tV3.instrument,stop=300)

2021-11-23 16:09:17.443185
2021-11-23 16:09:22.782717
1 2 3 4 5 6 7 8 9 10 
---------------------------------------------
2021-11-23T16:09:31.959952664Z | vado lungo
quantita  563.0 | prezzo:  1.1266 | profit/loss  0.0 | profitto totale=  0.0
-----------------------------------------------
11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 
---------------------------------------------
2021-11-23T16:12:01.623055497Z | chiudo posizione corta e vado lungo
quantita  1126.0 | prezzo:  1.12632 | profit/loss  0.0 | profitto totale=  0.0
-----------------------------------------------
146 147 148 149

In [12]:
tV4 = TraderV2("oanda.cfg","EUR_USD","30S",1,347,500)
print(datetime.utcnow()) # chiedo la data 
tV4.get_most_recent() 
print(datetime.utcnow())
tV4.stream_data(tV4.instrument,stop=300)

2021-11-23 16:14:36.221819
2021-11-23 16:14:41.775975
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 
---------------------------------------------
2021-11-23T16:15:00.601545028Z | vado lungo
quantita  563.0 | prezzo:  1.12638 | profit/loss  0.0 | profitto totale=  0.0
-----------------------------------------------
18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 
---------------------------------------------
2021-11-23T16:16:01.925469272Z | chiudo posizione lunga e vado corto
quantita  -1126.0 | prezzo:  1.12635 | profit/loss  -1.959 | profitto totale=  -1.959
-----------------------------------------------
72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 
---------------------------------------------
2021-11-23T16:16:30.437398399Z | chiudo posizione lunga e vado corto
quantita  -1126

###### IGNORARE:
 eh un blocco di lunghezza self period
 cose una candella: un blochetto di 5 secondi, 20 secondi, un tick??
 dove la "chiusura della candella" nel codice?
 questo: print(self.ticks, end=" ") -> stampa i numerini sullo schermo?
 questo metodo on_success viene chiamado dove? nel stream data della classe madre?
 Se devo calcolare logRet e posizione per ogni candella che arriva e non per tutto-ad ogni candella, 
 lunica soluzione magari eh rendere la classe iterativa? ma cosi devo cambiare tutto :-( 
 Iterativa vuol dire loop for i in range(0,len(df)) df["posizione"].iloc(i)= .....
 perche get most recent prende dati degli ultimi 5 giorni? Non basterebbe 1 giorno?
 
 
 
 # compro a ask, vendo a bid, altrimenti debbo simulare lo spread
    def get_most_recent(self,days=5):
        while True:
            now=datetime.utcnow()
            now = now -timedelta(microseconds=now.microsecond) #mi crea now togliendo i microsecondi
            yesterday = now - timedelta(days=days) #none ieri, sono days giorni prima
            df=self.get_history(self.instrument,start=yesterday,end=now,granularity="S5",price="M",localize=False).c.to_frame() 
            df["Abid"]=self.get_history(instrument=self.instrument, start=yesterday, end=now,granularity="S5",price="B", localize=False)["o"]
            df["Aask"]=self.get_history(instrument=self.instrument, start=yesterday, end=now,granularity="S5",price="A", localize=False)["o"]
            #df.rename(columns={"c":"EUR_USD"},inplace=True)
            self.rawData=df.resample(self.period,label="right").last().dropna().iloc[:-1] 
            #self.period devve essere piu grande di 5 
            self.ultimoPeriodo=self.rawData.index[-1] #potrebbe essere incompleto
            if (pd.to_datetime(datetime.utcnow()).tz_localize("UTC")-self.ultimoPeriodo)<pd.to_timedelta(self.period):
                break
            else:
                print("Purtroppo mi devo riscaricare i dati :(")
                time.sleep(2)